In [ ]:
%%writefile prepare.py
import os

os.system("python -m pip install authenticator pandas virtualenv && virtualenv env && . env/bin/activate && python -m pip install selenium pandas numpy && cd env/bin/ && wget -nc https://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_linux64.zip && unzip -u chromedriver_linux64.zip")

In [1]:
%%writefile tryMe.py
import os
import subprocess
import sys

try:
    import pandas as pd
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'pandas'])
finally:
    import pandas as pd

os.system("python -m pip install authenticator pandas virtualenv && virtualenv env && . env/bin/activate && python -m pip install selenium pandas numpy && cd env/bin/ && wget -nc https://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_linux64.zip && unzip -u chromedriver_linux64.zip")

data = pd.read_csv('store_credentials.csv', header=0, sep=';', index_col=False)
email = data.Email[0]
pswrd= data.Password[0]
shared_code = data.Shared[0]
os.system("echo 'yes\n"+pswrd +"\n"+pswrd+"\n"+shared_code+"\n"+"' | authenticator add Google:"+email+"; echo '#!/usr/bin/env bash\n(echo "+"'" +pswrd+"'"+" | authenticator generate) & sleep 1; kill $!' >> cook_otp; chmod +x cook_otp")

Writing tryMe.py


In [3]:
#%%writefile main.py
import os

os.system("touch test.py && touch prepare.py")
with open("test.py", "w") as f:   
    f.write("Hello World\nform " + f.name)       

#os.system("python prepare.py && python authentication.py && . env/bin/activate && python test.py && rm -r env")

In [ ]:
%%writefile authentication.py
import pandas as pd
import os
data = pd.read_csv('store_credentials.csv', header=0, sep=';', index_col=False)
email = data.Email[0]
pswrd= data.Password[0]
shared_code = data.Shared[0]
os.system("echo 'yes\n"+pswrd +"\n"+pswrd+"\n"+shared_code+"\n"+"' | authenticator add Google:"+email+"; echo '#!/usr/bin/env bash\n(echo "+"'" +pswrd+"'"+" | authenticator generate) & sleep 1; kill $!' >> cook_otp; chmod +x cook_otp")

In [ ]:
%%writefile test.py
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import selenium
import time
import pandas as pd
import os
import re
import numpy as np

data = pd.read_csv('store_credentials.csv', header=0, sep=';', index_col=False)
email = data.Email[0]
pswrd= data.Password[0]

OrdersColumn = np.array([])
BuyersColumn = np.array([])


chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome('env/bin/chromedriver',options=chrome_options)


scrape_url = "https://sellercentral.amazon.com/signin"
driver.get(scrape_url)
time.sleep(1)

try:
    username = driver.find_element_by_name("email")
    password =  driver.find_element_by_name("password")
    username.send_keys(email)
    password.send_keys(pswrd)
    driver.find_element_by_xpath("//input[@id='signInSubmit']").click()
    time.sleep(1)
    try:
        onetimepass = driver.find_element_by_name("otpCode")
        os.system('rm -f output.txt')
        os.system('echo "./cook_otp\nexit" | script output.txt') 
        with open("output.txt","r") as f:
            data = f.read()
            found = re.findall(r"(?<=com: ).*(?= \()",data,re.M)
            otp = found[0]
            print(otp)
            onetimepass.send_keys(otp)   
        driver.find_element_by_xpath("//input[@id='auth-signin-button']").click()
        time.sleep(1)
        driver.find_element_by_partial_link_text("Unshipped").click()
        time.sleep(2)
        
        steps = driver.find_elements_by_xpath("//li[@class='a-normal']")
        for step in steps:
            orders = driver.find_elements_by_xpath("//div[@class='cell-body']/div[@class='cell-body-title']")
            ords = []
            for order in orders:
                ords.append(order.text)
            OrdersColumn = np.append(OrdersColumn, np.array(ords))
            for ordi in ords:
                driver.find_element_by_partial_link_text(ordi).click()
                time.sleep(1)
                client = driver.find_element_by_xpath("//div[@data-test-id='shipping-section-buyer-address']")
                BuyersColumn = np.append(BuyersColumn, client.text)
                driver.execute_script("window.history.go(-1)")
                time.sleep(1)
            next_page = driver.find_element_by_xpath("//li[@class='a-last']")
            next_page.click()
            time.sleep(2)
        orders = driver.find_elements_by_xpath("//div[@class='cell-body']/div[@class='cell-body-title']")
        ords = []
        for order in orders:
            ords.append(order.text)
        OrdersColumn = np.append(OrdersColumn, np.array(ords))
        for ordi in ords:
            driver.find_element_by_partial_link_text(ordi).click()
            time.sleep(1)
            client = driver.find_element_by_xpath("//div[@data-test-id='shipping-section-buyer-address']")
            BuyersColumn = np.append(BuyersColumn, client.text)
            driver.execute_script("window.history.go(-1)")
            time.sleep(1)
    except selenium.common.exceptions.NoSuchElementException:
        print(driver.page_source)
except selenium.common.exceptions.WebDriverException:
    print(driver.page_source)
driver.close()

BuyersColumn = np.array([s.replace('\n', '. ', 1) for s in BuyersColumn])
BuyersColumn = np.array([s.replace('\n', ' ') for s in BuyersColumn])
spam = {'Orders':OrdersColumn, 'Buyers':BuyersColumn}
print(spam)
print(len(OrdersColumn))
print(len(BuyersColumn))
df = pd.DataFrame(spam, columns = ['Orders', 'Buyers'])
print(df)
df.to_csv('orders.csv',index=False)

In [ ]:
%%writefile main.py
import os

os.system("python prepare.py && python authentication.py && . env/bin/activate && python test.py && rm -r env")

In [ ]:
!python main.py